In [ ]:
import os
import re
import subprocess

import exiftool

import numpy as np

import utm

import rasterio
from rasterio.control import GroundControlPoint
from rasterio.transform import from_gcps

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
def parse_image_with_subprocess(image_path: str) -> dict:
    """
    Calls the exiftool command-line tool to extract metadata from an image
    and parses the resulting string output.

    Args:
        image_path: The full path to the image file.

    Returns:
        A dictionary containing the relevant parsed EXIF data.
    """
    # 1. Define the command to be executed.
    # It's a list where the first item is the program and the second is its argument.
    command = ["C:/Program Files/ExifTool/exiftool.exe", image_path]

    # Define the tags we want to keep
    relevant_tags = [
        "File Name",
        "Date/Time Original",
        "Camera Model Name",
        "Exif Image Width",
        "Exif Image Height",
        "Focal Length",
        "Focal Length In 35mm Format",
        "Field Of View",
        "GPS Latitude",
        "GPS Longitude",
        "Absolute Altitude",
        "Relative Altitude",
        "Gimbal Roll Degree",
        "Gimbal Yaw Degree",
        "Gimbal Pitch Degree",
    ]
    
    parsed_data = {}

    try:
        # 2. Run the command and capture its output.
        # check=True will raise an error if the command fails.
        # text=True decodes stdout/stderr as text.
        result = subprocess.run(
            command, 
            capture_output=True, 
            text=True, 
            check=True,
            encoding='utf-8' # Be explicit with encoding for cross-platform stability
        )
        exif_output_string = result.stdout

        # 3. Parse the captured string output line by line.
        for line in exif_output_string.strip().split('\n'):
            parts = line.split(':', 1)
            if len(parts) == 2:
                key = parts[0].strip()
                value = parts[1].strip()
                if key in relevant_tags:
                    parsed_data[key] = value

    except FileNotFoundError:
        print("Error: 'exiftool' not found. Is it installed and in your system's PATH?")
        return None
    except subprocess.CalledProcessError as e:
        print(f"Error running exiftool for {image_path}:")
        print(e.stderr) # Print the error output from exiftool
        return None
    
    return parsed_data


In [ ]:
def dms_to_dd(dms_str, ref):
    """Converts a DMS (Degrees, Minutes, Seconds) string to Decimal Degrees."""
    try:
        # Using regex to find numbers: handles various formats
        parts = re.findall(r'(\d+\.?\d*)', dms_str)
        degrees = float(parts[0])
        minutes = float(parts[1])
        seconds = float(parts[2])
        
        dd = degrees + minutes / 60.0 + seconds / 3600.0
        
        if ref in ['S', 'W']:
            dd *= -1
        return dd
    except (ValueError, IndexError) as e:
        print(f"Error parsing DMS string '{dms_str}': {e}")
        return None


def create_geotiff(parsed_data, source_jpg_path, output_tiff_path, compress=True, quality=85):
    """
    Creates a georeferenced TIFF from a JPG using its parsed EXIF metadata.
    """
    # --- Step A: Prepare data from the parsed dictionary ---
    try:
        # Image dimensions
        img_width = int(parsed_data['Exif Image Width'])
        img_height = int(parsed_data['Exif Image Height'])

        # Camera position
        lat_dd = dms_to_dd(parsed_data['GPS Latitude'], parsed_data['GPS Latitude'].split()[-1])
        lon_dd = dms_to_dd(parsed_data['GPS Longitude'], parsed_data['GPS Longitude'].split()[-1])
        # Use Absolute Altitude for sea level reference, Relative for AGL
        altitude_m = float(parsed_data['Absolute Altitude']) 
        
        # Camera orientation (gimbal)
        yaw_deg = float(parsed_data['Gimbal Yaw Degree'])
        pitch_deg = float(parsed_data['Gimbal Pitch Degree'])
        roll_deg = float(parsed_data['Gimbal Roll Degree'])

        # Camera intrinsics
        fov_deg = float(parsed_data['Field Of View'].split()[0])

    except (KeyError, ValueError) as e:
        print(f"Missing or invalid essential metadata: {e}")
        return

    # --- Step B: Convert camera position to UTM coordinates ---
    cam_x, cam_y, zone_num, zone_letter = utm.from_latlon(lat_dd, lon_dd)
    cam_z = altitude_m

    # --- Step C: Calculate ground coordinates of image corners ---
    # Convert angles to radians for numpy
    yaw = np.radians(yaw_deg)
    pitch = np.radians(pitch_deg)
    roll = np.radians(roll_deg)
    
    # Calculate ground footprint size assuming a flat plane at sea level (z=0)
    # This uses the altitude above the ground plane. If using absolute alt, this is an approximation.
    # For higher accuracy, you'd need a Digital Elevation Model (DEM).
    ground_height_m = 2 * cam_z * np.tan(np.radians(fov_deg / 2))
    aspect_ratio = img_width / img_height
    ground_width_m = ground_height_m * aspect_ratio

    # Define corners in the camera's coordinate system (X right, Y down, Z forward)
    # The image plane is effectively at a distance of `cam_z` from the ground
    half_w = ground_width_m / 2
    half_h = ground_height_m / 2
    
    # Define corners relative to the point directly below the camera
    corners_relative = np.array([
        [-half_w,  half_h, 0],  # Top-Left
        [ half_w,  half_h, 0],  # Top-Right
        [ half_w, -half_h, 0],  # Bottom-Right
        [-half_w, -half_h, 0]   # Bottom-Left
    ])

    # Create 3D rotation matrix (Yaw, Pitch, Roll)
    # Note: A pitch of -90 deg (downward) is the reference for a nadir image
    # We adjust pitch because our calculation is based on a nadir projection
    pitch_nadir_adjusted = np.radians(90 + pitch_deg)

    R_yaw = np.array([[np.cos(yaw), -np.sin(yaw), 0],
                      [np.sin(yaw),  np.cos(yaw), 0],
                      [0,           0,          1]])
    
    R_pitch = np.array([[np.cos(pitch_nadir_adjusted), 0, np.sin(pitch_nadir_adjusted)],
                        [0,                          1, 0],
                        [-np.sin(pitch_nadir_adjusted),0, np.cos(pitch_nadir_adjusted)]])
                      
    R_roll = np.array([[1, 0,           0],
                       [0, np.cos(roll), -np.sin(roll)],
                       [0, np.sin(roll),  np.cos(roll)]])
    
    R = R_yaw @ R_pitch @ R_roll # Combined rotation matrix

    # Rotate the corner points and add the camera's position
    rotated_corners = R @ corners_relative.T
    world_corners = rotated_corners.T + np.array([cam_x, cam_y, 0])

    # --- Step D: Create Ground Control Points (GCPs) ---
    pixel_corners = [
        (0, 0),                       # Top-Left
        (img_width, 0),               # Top-Right
        (img_width, img_height),      # Bottom-Right
        (0, img_height)               # Bottom-Left
    ]

    gcps = []
    for (col, row), (x, y, z) in zip(pixel_corners, world_corners):
        gcps.append(GroundControlPoint(row=row, col=col, x=x, y=y))

    # Calculate the affine transformation from the GCPs
    transform = from_gcps(gcps)

    # --- Step E: Write the georeferenced TIFF file (Corrected) ---
    try:
        with rasterio.open(source_jpg_path) as src:
            # Get the CRS for the UTM zone
            # Northern Hemisphere: 326xx, Southern Hemisphere: 327xx
            crs_epsg = f"EPSG:326{zone_num}" if lat_dd >= 0 else f"EPSG:327{zone_num}"
            
            # Read the source image's bands and metadata
            src_data = src.read()
            
            # Build the output profile from scratch
            profile = {
                'driver': 'GTiff',
                'height': img_height,
                'width': img_width,
                'count': src.count,      # Number of bands (e.g., 3 for RGB)
                'dtype': src.dtypes[0],  # Data type (e.g., uint8)
                'crs': crs_epsg,
                'transform': transform,
            }
            # If compression is enabled, update the profile
            if compress:
                # Check if the user wants high, lossy compression
                if quality < 100:
                    profile['compress'] = 'jpeg'
                    profile['jpeg_quality'] = quality
                    profile['photometric'] = 'YCBCR' # Recommended for JPEG compression
                else:
                    # Use a good lossless compression
                    profile['compress'] = 'lzw'

            # Write the new GeoTIFF file
            with rasterio.open(output_tiff_path, 'w', **profile) as dst:
                dst.write(src_data)
        
        print(f"✅ Successfully created georeferenced file: {output_tiff_path}")

    except Exception as e:
        print(f"Error writing GeoTIFF: {e}")


### Specify the Path

In [ ]:
src_path = os.path.abspath("../data/January_2024")

### Create GeoJSON file from Excel

In [ ]:
# 1. Get the list of Excel files in the source directory
xlsx_files = [os.path.join(src_path, f) for f in os.listdir(f"{src_path}/") if f.endswith(".xlsx")]
assert len(xlsx_files) == 1, "There should be exactly one Excel file in the directory."

xlsx_path = xlsx_files[0]
metadata_df = pd.read_excel(xlsx_path, header=None)

# 2. Find the row index that contains the actual headers
#    We search for a known column name like 'Latitude' to locate the row.
header_row_index = -1
for i, row in metadata_df.iterrows():
    # .astype(str) handles cases where some data might be numeric
    if 'Latitude' in row.astype(str).values:
        header_row_index = i
        break

if header_row_index == -1:
    raise ValueError("Could not find the header row containing 'Latitude'. Check the CSV file.")

# 3. Set the located row as the new column headers
metadata_df.columns = metadata_df.iloc[header_row_index]

# --- The rest of your data cleaning and geoprocessing script can now run reliably ---

# Force columns to numeric, coercing errors to NaN
metadata_df['Longitude'] = pd.to_numeric(metadata_df['Longitude'], errors='coerce')
metadata_df['Latitude'] = pd.to_numeric(metadata_df['Latitude'], errors='coerce')

# Drop any rows that failed conversion
metadata_df.dropna(subset=['Longitude', 'Latitude'], inplace=True)

# Create the GeoDataFrame
geometry = [Point(xy) for xy in zip(metadata_df['Longitude'], metadata_df['Latitude'])]
gdf = gpd.GeoDataFrame(metadata_df, geometry=geometry, crs="EPSG:4326")

# Save the final output
output_geojson_path = os.path.join(src_path, f"{os.path.splitext(os.path.basename(xlsx_path))[0]}.geojson")
gdf.to_file(output_geojson_path, driver='GeoJSON')

print(f"✅ Successfully cleaned data and created GeoJSON file at: {output_geojson_path}")

### Create "GeoReferenced" Images using EXIF data and Assumptions

In [ ]:
src_folder = os.path.abspath(f"{src_path}/images")
image_files = [f for f in os.listdir(src_folder) if f.lower().endswith((".jpg", ".jpeg", ".png"))]


for i_idx, image_file in enumerate(image_files):
    # Define the path to your image
    image_path = os.path.join(src_folder, image_file)
    output_path = ".".join(image_path.split(".")[:-1]) + ".TIF"
    
    if os.path.exists(output_path):
        print(f"File {output_path} already exists. Skipping...")
        continue
    
    # Parse the metadata
    parsed_metadata = parse_image_with_subprocess(image_path)

    # Create the GeoTIFF
    create_geotiff(parsed_metadata, image_path, output_path, compress=True)

